In [2]:
## Arxiv Research 
## tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [3]:
## use the inbuilt tools of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools = [wiki, arxiv]

In [6]:
## custom tool [RAG tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs =loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B443E44410>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [9]:
tools = [wiki, arxiv, retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\thaku\\Downloads\\m23\\ann_class_pro\\annenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B432642F20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

In [16]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [17]:
# prompt template
## using predesigned available template
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\thaku\Downloads\m23\ann_class_pro\annenv\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
## agents 
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [19]:
## agent executor 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools ,  verbose = True)

In [21]:
agent_executor.invoke({"input":"Tell me ab1out hritik roshan"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Hrithik Roshan'}`


Page: Hrithik Roshan
Summary: Hrithik Roshan (pronounced [ɾɪt̪ɪk ɾoʃən]; born 10 January 1974) is an Indian actor who works in Hindi cinema. He has portrayed a variety of characters and is known for his dancing skills. One of the highest-paid actors 
Invoking: `wikipedia` with `{'query': 'Hrithik Roshan films'}`


Page: Hrithik Roshan filmography
Summary: Hrithik Roshan is an Indian actor known for his work in Hindi-language films. As a child, he made uncredited appearances in three films directed by his maternal grandfather, J. Om Prakash, the first of which 
Invoking: `arxiv` with `{'query': 'Hrithik Roshan'}`


Published: 2024-11-07
Title: The Implicit Bias of Gradient Descent on Separable Multiclass Data
Authors: Hrithik Ravi, Clayton Scott, Daniel Soudry, Yutong Wang
Summary: Implicit bias describes the phenomenon where optimization-based training
algoritThe tool call id "call_yqnr"

{'input': 'Tell me ab1out hritik roshan',
 'output': 'The tool call id "call_yqnr" yielded a result that is not relevant to the topic of Hrithik Roshan, an Indian actor. Let\'s focus on the user\'s question.\n\nAs the user asked about Hrithik Roshan, I will provide a direct response.\n\nHrithik Roshan is a renowned Indian actor known for his versatility and dancing skills. He has appeared in numerous Bollywood films, including "Kaho Naa... Pyaar Hai", "Guru", "Zindagi Na Milegi Dobara", and "Kaabil", among many others. He has received several awards and nominations for his performances, including six Filmfare Awards.'}